In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mudata
from mudata import *

import numpy as np
import pandas as pd

# Dimensions
N = 100
D1, D2 = 10, 20
D = D1 + D2


In [3]:
mudata.set_options(pull_on_update=False)

In [3]:
def modalities(request, obs_n, obs_across, obs_mod):
    n_mod = 3
    mods = dict()
    np.random.seed(100)
    for i in range(n_mod):
        i1 = i + 1
        m = f"mod{i1}"
        mods[m] = AnnData(X=np.random.normal(size=1000 * i1).reshape(-1, 10 * i1))
        mods[m].obs["mod"] = m
        mods[m].var["mod"] = m

    if obs_n:
        if obs_n == "disjoint":
            mod2_which_obs = np.random.choice(
                mods["mod2"].obs_names, size=mods["mod2"].n_obs // 2, replace=False
            )
            mods["mod2"] = mods["mod2"][mod2_which_obs].copy()

    if obs_across:
        if obs_across != "intersecting":
            raise NotImplementedError("Tests for non-intersecting obs_names are not implemented")

    if obs_mod:
        if obs_mod == "duplicated":
            for m in ["mod1", "mod2"]:
                # Index does not support mutable operations
                obs_names = mods[m].obs_names.values.copy()
                obs_names[1] = obs_names[0]
                mods[m].obs_names = obs_names

    return mods

In [12]:
mods = dict(modalities(None, obs_n="disjoint", obs_across="intersecting", obs_mod="unique").items())

for m, mod in mods.items():
    # [mod1] var0, mod1_var1, mod1_var2, ...; [mod2] var0, mod2_var1, mod2_var2, ...
    mods[m].var_names = [f"{m}_var{j}" if j != 0 else f"var_{j}" for j in range(mod.n_vars)]
mdata = MuData(mods)
mdata.update()
mdata.pull_obs()
mdata.pull_var(join_common=False)


In [13]:
mdata.var

,mod1:mod,mod2:mod,mod3:mod
var_0,mod1,NaN,NaN
mod1_var1,mod1,NaN,NaN
mod1_var2,mod1,NaN,NaN
mod1_var3,mod1,NaN,NaN
mod1_var4,mod1,NaN,NaN
mod1_var5,mod1,NaN,NaN
mod1_var6,mod1,NaN,NaN
mod1_var7,mod1,NaN,NaN
mod1_var8,mod1,NaN,NaN
mod1_var9,mod1,NaN,NaN


In [6]:
mdata.var.head()

""
var0
var1
var2
var3
var4


In [7]:
mdata.pull_var()

In [8]:
mdata.var.head()

,assert-bool,mod1:assert-boolean-1,mod2:assert-boolean-2
var0,True,True,<NA>
var1,True,True,<NA>
var2,True,True,<NA>
var3,True,True,<NA>
var4,True,True,<NA>


In [10]:
assert mdata.var["assert-bool"].dtype == bool
assert isinstance(mdata.var["mod1:assert-boolean-1"].dtype, pd.BooleanDtype)
assert isinstance(mdata.var["mod2:assert-boolean-2"].dtype, pd.BooleanDtype)